## AmirMohammad_Ebrahiminasab - Real-Time-Facial-Recognition-Using-Fine_Tuned_VIT-Model

In [2]:
import cv2

In [3]:
import torch

### Importing a vit model structure in order to be able to update its weights to the previously fine tuned model.

In [4]:
from transformers import ViTForImageClassification, ViTFeatureExtractor


model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224-in21k', num_labels=7)
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\janja\anaconda3\lib\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


### Saving the structure so we don't have to download it each time.

In [5]:
torch.save(model.state_dict(), "C:\\Users\\janja\\Downloads\\pre_VIT_model.pth")

### Replacing the weights

In [7]:
add = torch.load("D:\\Career\\Real-Time-Facial-Recognition\\VIT_model.pth", map_location=torch.device('cpu'))

C:\Users\janja\AppData\Local\Temp\ipykernel_11852\2866471002.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  add = torch.load("D:\\Career\\Real-Time-Facial-Recognition\\

In [8]:
model.load_state_dict(add)

<All keys matched successfully>

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [10]:
from PIL import Image

In [11]:
emotions = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

### We use cascade for spotting the face and then we use the fine-tuned model for recognizing facial expressions and then showing the result using openCV.

In [12]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Initialize the webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        # Draw rectangle around the face
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

        # Extract the face region and resize for ViT input
        face = frame[y:y + h, x:x + w]
        face_resized = cv2.resize(face, (224, 224)) 

    
        pil_image = Image.fromarray(cv2.cvtColor(face_resized, cv2.COLOR_BGR2RGB))
        inputs = feature_extractor(images=pil_image, return_tensors="pt")

        # Forward pass through ViT model
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits

        # Get the predicted class
        predicted_class = torch.argmax(logits, dim=-1).item()

        # Display the predicted class label on the frame
        cv2.putText(frame, f"Class: {emotions[predicted_class]}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)

    # Show the frame with detection
    cv2.imshow('Real-Time Face Detection', frame)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()